In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np
from cnn_v1 import ConvNet_single
from dataloader_csv import CustomDatasetFromImages

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available(): 
    print("gpu availible")

In [ ]:
num_epochs,num_classes,batch_size,learning_rate = 30, 9,20,0.001

In [ ]:
train_path='../Dataset/Bytes/normal_train.csv'
val_path= '../Dataset/Bytes/normal_val.csv'

train_data = CustomDatasetFromImages(train_path)
val_data = CustomDatasetFromImages(val_path)

val_size = 500

train_data_loader = data.DataLoader(train_data, batch_size=batch_size,shuffle=True) 
val_data_loader  = data.DataLoader(val_data, batch_size=val_size, shuffle=True)

In [ ]:
model= ConvNet_single().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
path_to_checkpoint='../Dataset/Bytes/model/'
check_name='checkpoint.pth.tar'

def save_checkpoint(state, is_best, filename,loss):
    if is_best:
        print ("=> Saving a new lowest loss : "+str(loss))
        torch.save(state, filename)  # save checkpoint
        

In [ ]:
total_step = len(train_data_loader)
lowest_loss=0.800

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.to(device)
# Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        x=np.array((predicted==labels).cpu())
# Backward and optimize
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        val_loss_list=[]
        val_acc_list =[]
# Doing validation  
        for y,(images,labels) in enumerate(val_data_loader):
            images=images.to(device)
            labels=labels.to(device)
            outputs=model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_loss = criterion(outputs, labels)
            val_loss_list.append(val_loss.item())
            val_x=np.array((predicted==labels).cpu())
            val_acc=(sum(val_x))*100/len(val_x)
            val_acc_list.append(val_acc)
        mean_loss= np.mean(np.array(val_loss_list))
        mean_acc = np.mean(np.array(val_acc_list))
        is_best= bool(mean_loss<lowest_loss)
# If validation results are good then previous loss new state of the model will be save
        if(is_best):
            lowest_loss= min(mean_loss,lowest_loss)
            pathe= path_to_checkpoint+str(lowest_loss)+" "+str(epoch+1)+" "+check_name
            save_checkpoint({'epoch':epoch + 1,'state_dict': model.state_dict(),'lowest_loss': lowest_loss }, is_best,pathe,lowest_loss)
            is_best= False

        print("Epoch ["+str(epoch+1)+"/"+str(num_epochs)+"],Batch_no["+str(i+1)+"/"+str(total_step)+"] "+"Loss:"+str(format(loss.item(),'.4f'))+" Acc :"+str(format((sum(x))*100/len(x),'.4f'))+" Val_loss:"+str(format(mean_loss,'.4f'))+" Val_acc:"+str(format(mean_acc,'.0f')))